In [110]:
import pandas as pd
import datetime as dt

In [111]:
file = "immigration.xlsx"
df=pd.read_excel(file, engine='openpyxl',parse_dates=True, index_col=0)
df

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 121,Unnamed: 122,Unnamed: 123,Unnamed: 124,Unnamed: 125,Unnamed: 126,Unnamed: 127,Unnamed: 128,Unnamed: 129,Unnamed: 130
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Geography,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
date,Q1 1990,Q2 1990,Q3 1990,Q4 1990,Q1 1991,Q2 1991,Q3 1991,Q4 1991,Q1 1992,Q2 1992,...,Q1 2020,Q2 2020,Q3 2020,Q4 2020,Q1 2021,Q2 2021,Q3 2021,Q4 2021,Q1 2022,Q2 2022
NaN,Persons,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
population,43297,60119,62927,50081,51621,56753,54876,69526,56579,63300,...,69133,34070,40116,41267,70487,74438,122911,138190,113769,118114
Emigrants 4 5,7391,8840,14407,8974,9217,11094,17713,10006,8385,9529,...,9200,3371,8225,7065,10662,9886,15040,13137,11254,10338
Returning emigrants 5 6,2728,3960,5073,2840,2978,4317,5302,2968,3114,4515,...,6151,3319,13378,5539,5262,10754,17955,6268,5289,10814
Net temporary emigrants 5 7,..,..,..,..,..,..,6299,4632,4281,4529,...,7217,1896,3667,4254,5414,5507,7960,9531,6860,6595
Net non-permanent residents 8 9,17728,8986,4919,-42584,-18010,2822,2338,-41811,164,-3610,...,6462,-24348,-66091,-3151,14691,12174,57767,-38851,29012,157310


In [112]:
df= pd.DataFrame(df.iloc[[3,5], :]) 


In [113]:
df.reset_index(drop=True,inplace=True)
df

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 121,Unnamed: 122,Unnamed: 123,Unnamed: 124,Unnamed: 125,Unnamed: 126,Unnamed: 127,Unnamed: 128,Unnamed: 129,Unnamed: 130
0,Q1 1990,Q2 1990,Q3 1990,Q4 1990,Q1 1991,Q2 1991,Q3 1991,Q4 1991,Q1 1992,Q2 1992,...,Q1 2020,Q2 2020,Q3 2020,Q4 2020,Q1 2021,Q2 2021,Q3 2021,Q4 2021,Q1 2022,Q2 2022
1,43297,60119,62927,50081,51621,56753,54876,69526,56579,63300,...,69133,34070,40116,41267,70487,74438,122911,138190,113769,118114


In [114]:
#converting horizontal data to vertical
df1 = df.transpose()
df1.columns=["Date","immigrants"]
df1.reset_index(drop=True, inplace=True)
df1

,Date,immigrants
0,Q1 1990,43297
1,Q2 1990,60119
2,Q3 1990,62927
3,Q4 1990,50081
4,Q1 1991,51621
...,...,...
125,Q2 2021,74438
126,Q3 2021,122911
127,Q4 2021,138190
128,Q1 2022,113769


In [115]:
#converting string to datetime
def quarters_to_date(string):
    quarters = {'Q1':'01-01','Q2':'04-01','Q3':'07-01','Q4':'10-01'}
    quarter = string[0:2]
    year = string[3::]

    return f"{year}-{quarters.get(quarter)}"

df1['Date'] = df1['Date'].map(lambda x: quarters_to_date(x))

df1.head()



,Date,immigrants
0,1990-01-01,43297
1,1990-04-01,60119
2,1990-07-01,62927
3,1990-10-01,50081
4,1991-01-01,51621


In [116]:
df1.dtypes

Date          object
immigrants    object
dtype: object

In [117]:
#converting object data type to integer
df1["immigrants"]= df1["immigrants"].astype(object).astype(int)


In [118]:
#converting object type to datetime
df1["Date"] = pd.to_datetime(df1["Date"])


In [119]:
df1['Date'] = pd.to_datetime(df1['Date']).dt.to_period('m')
df1['immigrants'] /=  3

df1 = df1.iloc[df1.index.repeat(3)]

df1['Date'] = (df1['Date'].sub(df1.groupby(level=0)
                                    .cumcount(ascending=False))
                   .dt.to_timestamp(how='e').dt.normalize())

df1 = df1.reset_index(drop=True)



C:\Users\prans\AppData\Local\Temp\ipykernel_24100\3824998219.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Date'] = (df1['Date'].sub(df1.groupby(level=0)


In [120]:
df1["immigrants"] = df1["immigrants"].shift(2)

In [121]:
df1

,Date,immigrants
0,1989-11-30,NaN
1,1989-12-31,NaN
2,1990-01-31,14432.333333
3,1990-02-28,14432.333333
4,1990-03-31,14432.333333
...,...,...
385,2021-12-31,46063.333333
386,2022-01-31,37923.000000
387,2022-02-28,37923.000000
388,2022-03-31,37923.000000


In [122]:
df2 = df1.loc[ (df1['Date'] <='2021-12-31') & (df1['Date'] >= '2005-01-01')]
df2

,Date,immigrants
182,2005-01-31,18812.666667
183,2005-02-28,18812.666667
184,2005-03-31,18812.666667
185,2005-04-30,24823.666667
186,2005-05-31,24823.666667
...,...,...
381,2021-08-31,40970.333333
382,2021-09-30,40970.333333
383,2021-10-31,46063.333333
384,2021-11-30,46063.333333


In [123]:
#extracting only date and month from the date
df2['month_year'] = df2['Date'].dt.to_period('M')
df2

C:\Users\prans\AppData\Local\Temp\ipykernel_24100\566079457.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['month_year'] = df2['Date'].dt.to_period('M')


,Date,immigrants,month_year
182,2005-01-31,18812.666667,2005-01
183,2005-02-28,18812.666667,2005-02
184,2005-03-31,18812.666667,2005-03
185,2005-04-30,24823.666667,2005-04
186,2005-05-31,24823.666667,2005-05
...,...,...,...
381,2021-08-31,40970.333333,2021-08
382,2021-09-30,40970.333333,2021-09
383,2021-10-31,46063.333333,2021-10
384,2021-11-30,46063.333333,2021-11


In [124]:
df3 =df2.drop(columns=['Date'])


In [125]:
df3.rename(columns={'month_year':'date'})

,immigrants,date
182,18812.666667,2005-01
183,18812.666667,2005-02
184,18812.666667,2005-03
185,24823.666667,2005-04
186,24823.666667,2005-05
...,...,...
381,40970.333333,2021-08
382,40970.333333,2021-09
383,46063.333333,2021-10
384,46063.333333,2021-11


In [126]:
df3.dtypes

immigrants      float64
month_year    period[M]
dtype: object

In [127]:
df3.isnull().sum()

immigrants    0
month_year    0
dtype: int64